https://wiki.fysik.dtu.dk/ase/ase/calculators/espresso.html

https://github.com/Yavar-Azar/Courses/tree/main/QE/ASE-QE

In [ ]:
# ! export ASE_ESPRESSO_COMMAND="/path/to/pw.x -in PREFIX.pwi > PREFIX.pwo"

In [ ]:
from ase.io import read, write
from ase.calculators.espresso import Espresso

In [ ]:
crystal=read("MIN-167-350K-CuCbPyz.cif")

In [ ]:
inp_data={'prefix':"myprefix",'electron_maxstep':200,'outdir':"./",        'pseudo_dir':"./",'tstress':True,'tprnfor':True,'calculation':'scf', 
 'ecutrho':240,'verbosity':'high','ecutwfc':30, 'diagonalization': 'david', 'occupations':'smearing','smearing':'mp', 'mixing_mode':'plain', 'mixing_beta':0.7,'degauss':0.001, 'nspin':1}

# pseudodict={"Na":"", "As":"As.pbe-n-rrkjus_psl.1.0.0.UPF"}

In [ ]:
crystal.calc=Espresso(pseudopotentials=pseudodict,input_data=inp_data, kpts=(1,1,1))
crystal.get_potential_energy()

In [ ]:
write("input.pwi", crystal, pseudopotentials=pseudodict, input_data=inp_data, kpts=(1,1,1))

In [ ]:
# https://sites.google.com/site/dceresoli/pseudopotentials
pseudopotentials = {'Na': 'Na.pbe-tm-gipaw-dc.UPF',
                    'Cl': 'Cl.pbe-tm-gipaw.UPF'}

In [ ]:
import os
os.chdir('../..')
path = os.getcwd()
print(path)


In [ ]:
__file__

In [ ]:
os.path.abspath("")

In [ ]:
from ase.build import bulk
from ase.calculators.espresso import Espresso
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS


import os
from datetime import datetime
# os.makedirs('test', exist_ok=True)
# os.chdir('test')
dir_name = f'{datetime.now().strftime("%Y-%m-%d_%I %M-%S_%p")}'
dir_name = 'test/' + dir_name

os.makedirs(dir_name, exist_ok=False)
os.chdir(dir_name)


pseudo_dir = '../../Pseudopotentials'

rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)
# calc = Espresso(pseudopotentials=pseudopotentials, pseudo_dir = pseudo_dir,
#                 tstress=True, tprnfor=True, ecutwfc=10, kpts=(3, 3, 3))
# crystal.calc=Espresso(pseudopotentials=pseudodict,input_data=inp_data, kpts=(1,1,1))

calc = Espresso(pseudopotentials=pseudopotentials, pseudo_dir = pseudo_dir,
                tstress=True, tprnfor=True, ecutwfc=20, kpts=(3, 3, 3))

rocksalt.calc = calc

ucf = UnitCellFilter(rocksalt)
opt = LBFGS(ucf)
opt.run(fmax=0.005)

# cubic lattic constant
print((8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0))

os.chdir('../..')

In [ ]:
__file__